# Exercise 2.4

In [ ]:
import pandas as pd
import duckdb

df = pd.read_csv("data/landline-internet-subscriptions.csv")
df

,Entity,Code,Year,Fixed broadband subscriptions
0,Afghanistan,AFG,2004,200
1,Afghanistan,AFG,2005,220
2,Afghanistan,AFG,2006,500
3,Afghanistan,AFG,2007,500
4,Afghanistan,AFG,2008,500


In [103]:
norway_test = duckdb.query("""--sql
             SELECT *
             FROM df
             WHERE entity = 'Norway' AND year = 2023
    """).df()
norway_test[2023] = "no data"

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Entity                         4465 non-null   object
 1   Code                           4153 non-null   object
 2   Year                           4465 non-null   int64 
 3   Fixed broadband subscriptions  4465 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 139.7+ KB


In [206]:
data_2022 = duckdb.query("""--sql
    SELECT distinct entity, code, year, "Fixed broadband subscriptions"
    FROM df
    WHERE year = 2022 and entity NOT IN (SELECT entity FROM df WHERE year=2023)
                                 ORDER BY entity;
""").df()
data_2022

,Entity,Code,Year,Fixed broadband subscriptions
0,Antigua and Barbuda,ATG,2022,9920
1,Aruba,ABW,2022,18800
2,Barbados,BRB,2022,106000
3,Belize,BLZ,2022,39300
4,Bermuda,BMU,2022,23000
...,...,...,...,...
64,Tuvalu,TUV,2022,452
65,United States Virgin Islands,VIR,2022,8710
66,Vanuatu,VUT,2022,3530
67,Venezuela,VEN,2022,2700000


In [199]:
no_data_countries = duckdb.query("""--sql
    SELECT distinct entity, code
    FROM df
    WHERE entity NOT IN (SELECT entity FROM df WHERE year IN [2023, 2022])
                                 ORDER BY entity;
""").df()
no_data_countries

,Entity,Code
0,Sierra Leone,SLE


In [179]:
df_2023 = duckdb.query("""--sql
    SELECT *
    FROM df
    WHERE year=2023 and code != 'None'
    ORDER BY 4 DESC
""").df()
df_2023 = df_2023.iloc[1:]
df_2023

,Entity,Code,Year,Fixed broadband subscriptions
1,China,CHN,2023,636000000
2,United States,USA,2023,131000000
3,Brazil,BRA,2023,48400000
4,Japan,JPN,2023,47900000
5,India,IND,2023,39300000
...,...,...,...,...
136,Burundi,BDI,2023,2790
137,Palau,PLW,2023,1300
138,Kiribati,KIR,2023,440
139,South Sudan,SSD,2023,200


In [265]:
df_2023_full = pd.concat([df_2023, data_2022, no_data_countries])
df_2023_full["Fixed broadband subscriptions"] = df_2023_full["Fixed broadband subscriptions"].fillna(-1)
df_2023_full

,Entity,Code,Year,Fixed broadband subscriptions
1,China,CHN,2023.0,636000000.0
2,United States,USA,2023.0,131000000.0
3,Brazil,BRA,2023.0,48400000.0
4,Japan,JPN,2023.0,47900000.0
5,India,IND,2023.0,39300000.0
...,...,...,...,...
65,United States Virgin Islands,VIR,2022.0,8710.0
66,Vanuatu,VUT,2022.0,3530.0
67,Venezuela,VEN,2022.0,2700000.0
68,Yemen,YEM,2022.0,486000.0


In [213]:
import numpy as np

countries_log = np.log(df_2023_full["Fixed broadband subscriptions"])
countries_log

/Users/john.sandsjo/Documents/github/data-visualisation-john-sandsjo-de24/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

/Users/john.sandsjo/Documents/github/data-visualisation-john-sandsjo-de24/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

invalid value encountered in log



1     20.270709
2     18.690708
3     17.695010
4     17.684626
5     17.486735
        ...    
65     9.072227
66     8.169053
67    14.808762
68    13.093964
0           NaN
Name: Fixed broadband subscriptions, Length: 210, dtype: float64

In [318]:
import plotly.express as px

avg_internet = df_2023_full['Fixed broadband subscriptions'].max()*0.95
        

fig = px.choropleth(df_2023_full,
                    locations="Code",
                    color= countries_log,
                    hover_name="Entity",
                    color_continuous_scale="oranges",
                    custom_data=['Year', 'Fixed broadband subscriptions']
                    )

fig.update_layout(title="Landline Internet subscriptions, 2023",
                  title_subtitle_text="Subscriptions to fixed access to the public Internet with a download speed of at least 256 kbit/s.",
                  margin = dict(b=40, l=10, r=10),
                  width = 800,
                  coloraxis_colorbar=dict(
                    title=dict(text=""),
                    ticks="outside", orientation='h', xanchor='center',
                      yanchor='bottom',y=-0.3, len=0.8, thickness=10,
                        labelalias={20: "1 billion", 18: "300 million", 16: "100 million", 14: "30 million", 12: "10 million", 10: "3 million", 8: "1 million", 6: "300,000"}),
                  geo=dict(
                      showframe=False,
                      projection_type='natural earth'
                      ),
                  )

#fig.add_trace(colorbar=dict(orientation='h'))

fig.update_traces(hovertemplate="<b>%{hovertext}</b><br>" +
                                  "Year: %{customdata[0]}<br>" +
                                  "<b>Value: %{customdata[1]}</b>" 
                                  )

fig.add_annotation(
    text="<b>Data source:</b> International Telecommunication Union (via World Bank) (2025)",
    yref='paper', xref='paper',
    x=0, y=-0.1,  # Adjust y to position below the graph
    showarrow=False,
    font=dict(size=10)
)

fig.add_annotation(
    text="OurWorldInData.org/internet | CC BY",
    yref='paper', xref='paper',
    x=1, y=-0.1,  # Adjust y to position below the graph
    showarrow=False,
    font=dict(size=10)
)
fig.show()
fig.write_html("figures/2_4_internet_use.html")
